### Let's Code a Simple Question Answering System with OpenAI
This hands-on project will guide you through the process of programming, covering essential steps such as defining prompt templates, integrating OPEN AI as language model, and implementing retrieval mechanisms to fetch information. By the end of the section, you'll have gained practical experience in building a simple yet effective question-answering solution, offering valuable insights into the mechanics of this exciting field.

In [ ]:
pip install langchain lxml chromadb sentence-transformers ctransformers openai

### Load the dataset
Visit Wikipedia and retrieve the [Wikipedia page for the Porsche 911](https://en.wikipedia.org/wiki/Porsche_911). In this simplified example, we are only loading a single page, but in practice, you have the capability to load multiple pages.

In [2]:
from langchain.text_splitter import HTMLHeaderTextSplitter

file_path= r'.\data\Wikipedia 911\Porsche 911 - Wikipedia.html'
file1 = open(file_path, encoding='utf-8')

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("h5", "Header 5"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on, return_each_element=None)
html_header_splits = html_splitter.split_text(file1.read())
print("Loader {} files".format(len(html_header_splits)))

Loader 44 files


### Split documents and save them into a vector database
In the subsequent phase, we divide the HTML page into subsections according to the headers. Feel free to employ alternative criteria for segmentation.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from chromadb.utils import embedding_functions
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# Define the Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 384,
    chunk_overlap = 50
)

#Create a split of the document using the text splitter
splits = text_splitter.split_documents(html_header_splits)

### Create and Store embeddings in a vector database.
Next, we save all the data as word embeddings in ChromaDB. The embedding function() utilizes a transformer model loaded from the SBert library. You have the flexibility to experiment with various models. For reference, check the available options in the Pretrained Models section of the Sentence-Transformers documentation at [sbert.net](https://www.sbert.net/).

In [ ]:
# Embedd the Splits
default_ef = embedding_functions.DefaultEmbeddingFunction()
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function, #embedding,
)

print(f'{vectordb._collection.count()} Embeddings are loaded in the Vector Database')

### Initialize the OpenAI Model

Load a OpenAI Model to answer the question. This can take a few minutes because the model will be downloaded the first time. 

Make sure you have and [OpenAI Account](https://platform.openai.com/assistants) and you created an API-Key. Make sure you
create an ENV variable called 

- OPENAI_API_KEY

with the created API-Key to use OpenAI. More information [here](https://platform.openai.com/docs/quickstart?context=python)

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm=OpenAI()

def setupTheQAChain(modelType = "gpt-3.5-turbo"):

    print("Using model: {}".format(modelType))

    llm = ChatOpenAI(model_name=modelType, temperature=0)

    template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.  
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
        return_source_documents=True,
        chain_type='stuff',
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    return qa_chain

### Let's bring everything together

In [12]:
qa_chain = setupTheQAChain()
results = qa_chain.invoke({"query": "Which Porsche has the highest top speed?" })
print('Query: {} \nResults {} \nSource: {}'.format(results['query'], results['result'], results['source_documents']))

Using model: gpt-3.5-turbo
Query: Which Porsche has the highest top speed? 
Results The Porsche 911 GT1 has the highest top speed of 310 km/h (193 mph). 
Source: [Document(page_content='Unlike the previous GT2 versions, this car is fitted with a 7-speed PDK transmission to handle the excessive torque produced from the engine. Porsche claims that this car will accelerate from 0 to 60\xa0mph (97\xa0km/h) in 2.7\xa0seconds, and has a top speed of 340\xa0km/h (210\xa0mph).', metadata={'Header 1': 'Porsche 911', 'Header 2': 'Water-cooled engines (1998–present)[edit]', 'Header 3': '991 Series (2011 – 2019)[edit]', 'Header 4': '991 GT2 RS[edit]'}), Document(page_content='magazine tested the 997 GT3 and recorded a 0–100\xa0km/h (62\xa0mph) of 3.9\xa0seconds and a top speed of 312\xa0km/h (194\xa0mph). It was at that time crowned "the best handling car in America" by Motor Trend.[citation needed]', metadata={'Header 1': 'Porsche 911', 'Header 2': 'Water-cooled engines (1998–present)[edit]', 'He